In [1]:
import cv2
import io
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import net_sphere
import torch.optim as optim
from torch.nn.utils import clip_grad_norm
from skimage import io
import numpy as np
import datetime,sys
from numpy.random import randint
from calculateEvaluationCCC import calculateCCC

# Define parameters
use_cuda = torch.cuda.is_available()
# lr = 0.01
# bs = 32
# n_epoch = 20
# lr_steps = [7,14]
lr = 0.01
bs = 32
n_epoch = 30
lr_steps = [8,16,24]

gd = 20 # clip gradient
eval_freq = 3
print_freq = 20
num_worker = 4
num_seg = 16
flag_biLSTM = True

classnum = 7

train_list_path = './data/OMG_Aligned/train_list_lstm.txt'
val_list_path = './data/OMG_Aligned/val_list_lstm.txt'
model_path = './model/sphere20a_20171020.pth'

In [2]:
sphereface = getattr(net_sphere,'sphere20a')()
sphereface.load_state_dict(torch.load(model_path))
sphereface.feature = True # remove the last fc layer because we need to use LSTM first

class Net(torch.nn.Module):
    def __init__(self, sphereface):
        super(Net, self).__init__()
        self.sphereface = sphereface
        self.linear = torch.nn.Linear(512,2)
        self.tanh = torch.nn.Tanh()
        self.avgPool = torch.nn.AvgPool2d((num_seg,1), stride=1)
        self.LSTM = torch.nn.LSTM(512, 512, 1, batch_first = True, dropout=0.2, bidirectional=flag_biLSTM)  # Input dim, hidden dim, num_layer
        for name, param in self.LSTM.named_parameters():
            if 'bias' in name:
                torch.nn.init.constant(param, 0.0)
            elif 'weight' in name:
                torch.nn.init.orthogonal(param)
        
    def sequentialLSTM(self, input, hidden=None):

        input_lstm = input.view([-1,num_seg, input.shape[1]])
        batch_size = input_lstm.shape[0]
        feature_size = input_lstm.shape[2]

        self.LSTM.flatten_parameters()
            
        output_lstm, hidden = self.LSTM(input_lstm)
        if flag_biLSTM:
             output_lstm = output_lstm.contiguous().view(batch_size, output_lstm.size(1), 2, -1).sum(2).view(batch_size, output_lstm.size(1), -1) 

        # avarage the output of LSTM
#         print '0', output_lstm.shape
        output_lstm = output_lstm.view(batch_size,1,num_seg,-1)
#         print '1',output_lstm.shape
        out = self.avgPool(output_lstm)
#         print '2',out.shape
        out = out.view(batch_size,-1)
        return out
    
    def forward(self, x):
        x = self.sphereface(x)
        x = self.sequentialLSTM(x)
        x = self.linear(x)
        x = self.tanh(x)
        
        return x

model = Net(sphereface)

if use_cuda:
    model.cuda()
    
    
def 

criterion = torch.nn.MSELoss()

In [3]:
class OMGDataset(Dataset):
    """OMG dataset."""

    def __init__(self, txt_file, base_path, transform=None):
        self.base_path = base_path
        self.data = pd.read_csv(txt_file, sep=" ", header=None)
#         self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        vid = self.data.iloc[idx,0]
        utter = self.data.iloc[idx,1]
        img_list = self.data.iloc[idx,-1]
        img_list = img_list.split(',')[:-1]
        img_list = map(int, img_list)
        
        num_frames = len(img_list)
        # inspired by TSN's pytorch code
        average_duration = num_frames // num_seg
        if num_frames>num_seg:
            offsets = np.multiply(list(range(num_seg)), average_duration) + randint(average_duration, size=num_seg)
        else:
            tick = num_frames / float(num_seg)
            offsets = np.array([int(tick / 2.0 + tick * x) for x in range(num_seg)])

        final_list = [img_list[i] for i in offsets]
        
        # stack images within a video in the depth dimension
        for i,ind in enumerate(final_list):
            image = io.imread(self.base_path+'%s/%s/%d.jpg'%(vid,utter,ind)).astype(np.float32)
            image = torch.from_numpy(((image - 127.5)/128).transpose(2,0,1))
            if i==0:
                images = image
            else:
                images = torch.cat((images,image), 0)
        
        
        label = torch.from_numpy(np.array([self.data.iloc[idx,2], self.data.iloc[idx,3]]).astype(np.float32))

        if self.transform:
            image = self.transform(image)
        return (images, label, (vid,utter))
    
train_loader = DataLoader(OMGDataset(train_list_path,'./data/OMG_Aligned/Train/'), 
                          batch_size=bs, shuffle=True, num_workers=num_worker)
val_loader = DataLoader(OMGDataset(val_list_path,'./data/OMG_Aligned/Val/'), 
                        batch_size=bs, shuffle=False, num_workers=num_worker)



In [4]:
def printoneline(*argv):
    s = ''
    for arg in argv: s += str(arg) + ' '
    s = s[:-1]
    sys.stdout.write('\r'+s)
    sys.stdout.flush()
    
def dt():
    return datetime.datetime.now().strftime('%H:%M:%S')

def save_model(model,filename):
    state = model.state_dict()
#     for key in state: state[key] = state[key].clone().cpu()
    torch.save(state, filename)

In [5]:
def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    
    train_loss = 0
    correct = 0
    total = 0
    batch_idx = 0
    
    for i, (inputs, targets, _) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda(async=True)

        inputs = torch.autograd.Variable(inputs)
        targets = torch.autograd.Variable(targets)
        
        inputs = inputs.view((-1,3)+inputs.size()[-2:])
        outputs = model(inputs)
        
#         print 'here',outputs.shape, targets.shape
        
        loss = criterion(outputs,targets)
        
#         lossd = loss.data[0]        
        loss.backward()
        optimizer.step()
        
        #tsn uses clipping gradient
        if gd is not None:
            total_norm = clip_grad_norm(model.parameters(),gd)
            if total_norm > gd:
                print('clippling gradient: {} with coef {}'.format(total_norm, gd/total_norm))
                
        train_loss += loss.data[0]
        
        if i % print_freq == 0:
            printoneline(dt(),'Epoch=%d Loss=%.4f\n'
                % (epoch,train_loss/(batch_idx+1)))
        batch_idx += 1


def validate(val_loader, model, criterion, epoch):
    model.eval()
    
    err_arou = 0.0
    err_vale = 0.0
    
    txt_result = open('results/val_lstm_%d.csv'%epoch, 'w')
    txt_result.write('video,utterance,arousal,valence\n')
    for (inputs, targets,(vid, utter)) in val_loader:
#         print inputs.shape
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        
        inputs = torch.autograd.Variable(inputs)
        targets = torch.autograd.Variable(targets)
        
        inputs = inputs.view((-1,3)+inputs.size()[-2:])
#         print inputs.size()[-2:]
#         print inputs.shape
        outputs = model(inputs)
        
#         print outputs[:,0]-targets[:,0]
        outputs = outputs.data.cpu().numpy()
        targets = targets.data.cpu().numpy()
        
#         print outputs, targets
        err_arou += np.sum((outputs[:,0]-targets[:,0])**2)
        err_vale += np.sum((outputs[:,1]-targets[:,1])**2)
        
        for i in range(len(vid)):
#             name = img_name[i].replace('/home/m2a03/Work/sphereface_pytorch/data/OMG_Aligned/', '')
            out = outputs
#             print vid[0], utter[0]
            txt_result.write('%s,%s.mp4,%f,%f\n'%(vid[i], utter[i],out[i][0],out[i][1]))
    
#     print('MSE of arousal: %f' % (err_arou / len(val_loader.dataset)))
#     print('MSE of valence: %f' % (err_vale / len(val_loader.dataset)))
#     print('MSE of total: %f' % ((err_arou + err_vale) / len(val_loader.dataset)))
    txt_result.close()
    
    arouCCC, valeCCC = calculateCCC('./results/omg_ValidationVideos.csv','results/val_lstm_%d.csv'%epoch)
    return (arouCCC,valeCCC)



    
    
        

In [6]:
# best_mse = validate(val_loader, model, criterion, 0)

# print best_mse

In [7]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)    
best_arou_ccc, best_vale_ccc = validate(val_loader, model, criterion,0)
for epoch in range(n_epoch):
    if epoch in lr_steps:
        lr *= 0.1
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)    

    train(train_loader, model, criterion, optimizer, epoch)
#     save_model(model, './results/epoch_lstm_{}.pth'.format(epoch))
    
    # evaluate on validation set
    if (epoch+1)%eval_freq == 0 or epoch == n_epoch-1:
        arou_ccc, vale_ccc = validate(val_loader, model, criterion,epoch)
        
        if (arou_ccc+vale_ccc) > (best_arou_ccc + best_vale_ccc):
            best_arou_ccc = arou_ccc
            best_vale_ccc = vale_ccc
            save_model(model,'./pth/model_lstm_{}_{}_{}.pth'.format(epoch, round(arou_ccc,4), round(vale_ccc,4)))
            

Arousal CCC:  -0.006611498662120568
Arousal Cor:  -0.03837046335237764
Arousal MSE:  0.342062354754905
Valence CCC:  0.009066932179317127
Valence Cor:  0.015659614804120568
Valence MSE:  0.20248923395417961
Total CCC:    0.002455433517196559
10:51:27 Epoch=0 Loss=0.2141
10:52:16 Epoch=0 Loss=0.1206
10:53:06 Epoch=0 Loss=0.0961
10:53:55 Epoch=0 Loss=0.0840
10:54:36 Epoch=1 Loss=0.0661
10:55:26 Epoch=1 Loss=0.0513
10:56:15 Epoch=1 Loss=0.0497
10:57:05 Epoch=1 Loss=0.0486
10:57:46 Epoch=2 Loss=0.0303
10:58:35 Epoch=2 Loss=0.0403
10:59:25 Epoch=2 Loss=0.0418
11:00:14 Epoch=2 Loss=0.0410
Arousal CCC:  0.1992311863391338
Arousal Cor:  0.23305603583524825
Arousal MSE:  0.051668559858100746
Valence CCC:  0.3640836374604136
Valence Cor:  0.3789696962662488
Valence MSE:  0.11537642154812851
Total CCC:    0.5633148237995474
11:01:10 Epoch=3 Loss=0.0294
11:02:00 Epoch=3 Loss=0.0370
11:02:49 Epoch=3 Loss=0.0365
11:03:39 Epoch=3 Loss=0.0377
11:04:19 Epoch=4 Loss=0.0411
11:05:09 Epoch=4 Loss=0.0335
1

Process Process-106:
Process Process-107:
Process Process-105:
Process Process-108:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/m2a03/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Traceback (most recent call last):
    self.run()
  File "/home/m2a03/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/m2a03/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
Traceback (most recent call last):
    self._target(*self._args, **self._kwargs)
  File "/home/m2a03/anaconda2/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
  File "/home/m2a03/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/m2a03/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    r = index_queue.get()
    self.run()
    self._target(*self._args, **self._kwargs)
  File "/home/m2a03/anacon

KeyboardInterrupt: 

In [ ]:
# t = '/home/m2a03/Work/sphereface_pytorch/data/OMG_Aligned/Val/74de88564/utterance_1/0.jpg'
# tt = t.replace('/home/m2a03/Work/sphereface_pytorch/data/OMG_Aligned/','')
# print tt